In [1]:
# preprocess_shared.ipynb
import os
import numpy as np
import pandas as pd
import pyxdf
from dataclasses import dataclass
from scipy.signal import butter, filtfilt

# -------------------------
# Config
# -------------------------
@dataclass
class PreprocConfig:
    # XDF search roots
    xdf_root_cf: str
    xdf_root_sd: str

    # Streams
    emg_stream_name: str = "EMG"
    com_stream_name: str = "CenterOfMass1"
    video_stream_names: tuple = ("video data", "video_data")

    # Kinematics (optional)
    kin_stream_name: str = "LinearSegmentKinematicsDatagram1"
    include_kin_acc_features: bool = True   # keeps backward compatibility
    kin_segments: tuple = (1, 5, 7)          # Pelvis=1, T8=5, Head=7 (MATLAB indexing)
    kin_strict: bool = True                 # if True, raise when kinematics stream is missing

    # Resampling
    target_fs: float = 50.0
    min_trial_duration_sec: float = 5.0

    # EMG envelope
    emg_bandpass_hz: tuple = (10.0, 400.0)
    emg_envelope_lp_hz: float = 8.0

    # MoS outlier filtering
    b_cm_abs_thresh: float = 100.0
    outlier_ratio_max: float = 0.1

    # Misstep event tolerance band
    event_tol_pre_sec: float = 0.05
    event_tol_post_sec: float = 0.10


# -------------------------
# General helpers
# -------------------------
def normalize_trial_name(name: str) -> str:
    s = str(name).strip()
    if not s:
        return ""
    if not s.lower().endswith(".xdf"):
        s += ".xdf"
    return s

def find_xdf_path(trial_xdf: str, cfg: PreprocConfig) -> str:
    p1 = os.path.join(cfg.xdf_root_cf, trial_xdf)
    if os.path.exists(p1):
        return p1
    p2 = os.path.join(cfg.xdf_root_sd, trial_xdf)
    if os.path.exists(p2):
        return p2
    return ""

def estimate_fs(ts: np.ndarray) -> float:
    ts = np.asarray(ts, dtype=float)
    dt = np.diff(ts)
    dt = dt[dt > 0]
    if len(dt) == 0:
        return np.nan
    return 1.0 / np.median(dt)

def parse_float_from_cell(x):
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return np.nan
    if isinstance(x, (int, float, np.integer, np.floating)):
        return float(x)
    s = str(x).strip()
    if not s:
        return np.nan
    try:
        return float(s)
    except Exception:
        return np.nan

def parse_type_from_cell(x):
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return "UNK"
    s = str(x).strip().upper()
    if s in ("F", "B", "FB"):
        return s
    if s == "":
        return "UNK"
    return s

# -------------------------
# Stream loaders
# -------------------------
def load_stream_df(xdf_path: str, stream_name: str, col_prefix: str) -> pd.DataFrame | None:
    streams, _ = pyxdf.load_xdf(xdf_path)
    target = None
    for s in streams:
        if s["info"]["name"][0] == stream_name:
            target = s
            break
    if target is None:
        return None

    ts = np.asarray(target["time_stamps"], dtype=float)
    data = np.asarray(target["time_series"])
    if data.ndim == 1:
        data = data.reshape(-1, 1)

    df = pd.DataFrame(data, columns=[f"{col_prefix}{i+1}" for i in range(data.shape[1])])
    df["timestamp"] = ts
    df = df.sort_values("timestamp").reset_index(drop=True)
    return df

def _kin_col_name(col_prefix: str, one_based_idx: int) -> str:
    return f"{col_prefix}{one_based_idx}"

def compute_kin_acc_mag_df(kin_df: pd.DataFrame, cfg: PreprocConfig, col_prefix: str = "KIN_ch") -> pd.DataFrame:
    """
    Compute acceleration magnitude for selected segments from a kinematics stream that follows
    the MATLAB convention of 9 values per segment (pos[1:3], vel[4:6], acc[7:9]) across 23 segments.

    Expects columns like: KIN_ch1..KIN_ch207 plus 'timestamp'.
    Returns DataFrame with 'timestamp' and columns:
      kin_acc_mag_seg{segIdx} (one per cfg.kin_segments)
    """
    if kin_df is None or len(kin_df) == 0:
        return None

    ts = kin_df["timestamp"].values.astype(float)
    out = {"timestamp": ts}

    # Ensure we have the expected columns; if not, we will still try gracefully.
    for seg_idx in cfg.kin_segments:
        base = (int(seg_idx) - 1) * 9  # 0-based offset in MATLAB rows
        ax_col = _kin_col_name(col_prefix, base + 7)
        ay_col = _kin_col_name(col_prefix, base + 8)
        az_col = _kin_col_name(col_prefix, base + 9)

        if ax_col not in kin_df.columns or ay_col not in kin_df.columns or az_col not in kin_df.columns:
            # missing expected channels -> produce NaNs so downstream interpolation works
            out[f"kin_acc_mag_seg{seg_idx}"] = np.full_like(ts, np.nan, dtype=float)
            continue

        ax = kin_df[ax_col].values.astype(float)
        ay = kin_df[ay_col].values.astype(float)
        az = kin_df[az_col].values.astype(float)
        out[f"kin_acc_mag_seg{seg_idx}"] = np.sqrt(ax*ax + ay*ay + az*az)

    return pd.DataFrame(out).sort_values("timestamp").reset_index(drop=True)


def load_kin_df(xdf_path: str, cfg: PreprocConfig) -> pd.DataFrame | None:
    """Convenience loader for the kinematics stream."""
    return load_stream_df(xdf_path, cfg.kin_stream_name, col_prefix="KIN_ch")



def clean_emg_envelope(df_emg: pd.DataFrame, cfg: PreprocConfig) -> pd.DataFrame:
    df = df_emg.copy()
    ts = df["timestamp"].values.astype(float)
    fs = estimate_fs(ts)
    if not np.isfinite(fs) or fs <= 0:
        return df

    nyq = 0.5 * fs
    low_hz, high_hz = cfg.emg_bandpass_hz
    low = max(low_hz / nyq, 0.001)
    high = min(high_hz / nyq, 0.99)

    if high > low:
        b_bp, a_bp = butter(2, [low, high], btype="band")
    else:
        b_bp, a_bp = butter(2, low, btype="high")

    wn = min(max(cfg.emg_envelope_lp_hz / nyq, 0.001), 0.99)
    b_lp, a_lp = butter(4, wn, btype="low")

    emg_cols = [c for c in df.columns if c.startswith("EMG_ch")]
    for c in emg_cols:
        sig = df[c].values.astype(float)
        # guard filtfilt against NaNs
        if not np.isfinite(sig).all():
            sig = pd.Series(sig).interpolate(limit_direction="both").values
        try:
            sig_f = filtfilt(b_bp, a_bp, sig)
            sig_r = np.abs(sig_f)
            sig_e = filtfilt(b_lp, a_lp, sig_r)
            df[c] = sig_e.astype(np.float32)
        except Exception:
            df[c] = sig.astype(np.float32)

    return df

def load_video_df(xdf_path: str, cfg: PreprocConfig) -> pd.DataFrame | None:
    streams, _ = pyxdf.load_xdf(xdf_path)
    video = None
    for s in streams:
        nm = s["info"]["name"][0]
        if nm in cfg.video_stream_names:
            video = s
            break
    if video is None:
        return None

    ts = np.asarray(video["time_stamps"], dtype=float)
    data = np.asarray(video["time_series"])

    if data.ndim == 1:
        frame = data.astype(float)
        t_video_s = ts - ts[0]
    else:
        frame = data[:, 0].astype(float)
        if data.shape[1] >= 2:
            raw_sec = data[:, 1]
            t_video_s = np.array([float(str(x).split()[0]) for x in raw_sec], dtype=float)
        else:
            t_video_s = ts - ts[0]

    df = pd.DataFrame({"timestamp": ts, "frame": frame, "t_video_s": t_video_s})
    df = df.sort_values("timestamp").reset_index(drop=True)
    return df

# -------------------------
# MoS loaders + filters
# -------------------------
def compute_keep_files(b_csv_path: str, cfg: PreprocConfig) -> set:
    df = pd.read_csv(b_csv_path)
    if "file_name" not in df.columns or "b_cm" not in df.columns:
        raise ValueError("b_csv must contain columns: file_name, b_cm")

    mask = df["b_cm"].abs() > cfg.b_cm_abs_thresh
    ratio = mask.groupby(df["file_name"]).mean()
    keep = set(ratio[ratio <= cfg.outlier_ratio_max].index.tolist())
    return keep

def _mos_time_col(df: pd.DataFrame) -> str:
    for c in ["time_s", "timestamp", "time"]:
        if c in df.columns:
            return c
    raise ValueError("b_csv must contain one of: time_s / timestamp / time")

def build_mos_dict(b_csv_path: str, trial_keys: list[str], cfg: PreprocConfig) -> dict:
    df = pd.read_csv(b_csv_path)
    time_col = _mos_time_col(df)
    df = df.rename(columns={time_col: "timestamp"})

    mos_dict = {}
    for fname in trial_keys:
        sub = df[df["file_name"].astype(str) == str(fname)].copy()
        if sub.empty:
            continue

        # outlier clip for b_cm
        sub = sub[sub["b_cm"].abs() <= cfg.b_cm_abs_thresh].copy()
        sub = sub.sort_values("timestamp").reset_index(drop=True)
        if sub.empty:
            continue

        keep_cols = ["timestamp", "b_cm"]
        if "b_m" in sub.columns:
            keep_cols.append("b_m")
        sub = sub[keep_cols].copy()

        for c in keep_cols:
            sub[c] = pd.to_numeric(sub[c], errors="coerce")

        sub = sub.dropna(subset=["timestamp", "b_cm"]).reset_index(drop=True)
        if sub.empty:
            continue

        mos_dict[fname] = sub

    return mos_dict

def load_mos_df_for_trial(b_csv_path: str, trial_xdf: str, cfg: PreprocConfig) -> pd.DataFrame | None:
    df = pd.read_csv(b_csv_path)
    time_col = _mos_time_col(df)

    if "file_name" not in df.columns or "b_cm" not in df.columns:
        raise ValueError("MoS CSV must include: file_name, b_cm (and optionally b_m)")

    sub = df[df["file_name"].astype(str) == str(trial_xdf)].copy()
    if sub.empty:
        return None

    sub = sub.rename(columns={time_col: "timestamp"}).sort_values("timestamp").reset_index(drop=True)

    keep_cols = ["timestamp", "b_cm"]
    if "b_m" in sub.columns:
        keep_cols.append("b_m")
    sub = sub[keep_cols].copy()

    for c in keep_cols:
        sub[c] = pd.to_numeric(sub[c], errors="coerce")
    sub = sub.dropna(subset=["timestamp", "b_cm"]).reset_index(drop=True)
    return sub if not sub.empty else None

# -------------------------
# Uniform feature construction (used by BOTH builder + inference)
# -------------------------
def build_uniform_features(mos_df: pd.DataFrame, emg_df: pd.DataFrame, com_df: pd.DataFrame, cfg: PreprocConfig, kin_df: pd.DataFrame | None = None):
    """
    Returns:
      t_uniform (abs time)
      feat_df columns: timestamp, b_cm, b_m(optional), EMG_ch*_env, COM_ch*
      feature_names list in exact order for model input (excluding timestamp)
    """
    mos_ts = mos_df["timestamp"].values.astype(float)
    t0, t1 = float(mos_ts[0]), float(mos_ts[-1])

    if (t1 - t0) < cfg.min_trial_duration_sec:
        return None, None, None

    dt = 1.0 / float(cfg.target_fs)
    t_uniform = np.arange(t0, t1, dt)

    emg_ts = emg_df["timestamp"].values.astype(float)
    com_ts = com_df["timestamp"].values.astype(float)

    cols = {"timestamp": t_uniform}
    feature_names = []

    # MoS first
    cols["b_cm"] = np.interp(t_uniform, mos_ts, mos_df["b_cm"].values.astype(float))
    feature_names.append("b_cm")
    if "b_m" in mos_df.columns:
        cols["b_m"] = np.interp(t_uniform, mos_ts, mos_df["b_m"].values.astype(float))
        feature_names.append("b_m")

    # EMG channels (assumes emg_df already envelope-filtered)
    emg_cols = [c for c in emg_df.columns if c.startswith("EMG_ch")]
    for c in emg_cols:
        colname = f"{c}_env"
        cols[colname] = np.interp(t_uniform, emg_ts, emg_df[c].values.astype(float))
        feature_names.append(colname)

    
    # Kinematics accel magnitude (optional; appended AFTER EMG, BEFORE COM to keep old ordering unless enabled)
    if getattr(cfg, "include_kin_acc_features", False) and (kin_df is not None) and len(kin_df) > 0:
        kin_mag_df = compute_kin_acc_mag_df(kin_df, cfg, col_prefix="KIN_ch")
        kin_ts = kin_mag_df["timestamp"].values.astype(float)

        # Map MATLAB segment indices to friendly names (fixed mapping for (1,5,7))
        friendly = {1: "Pelvis", 5: "T8", 7: "Head"}
        for seg_idx in cfg.kin_segments:
            src_col = f"kin_acc_mag_seg{int(seg_idx)}"
            name = friendly.get(int(seg_idx), f"Seg{int(seg_idx)}")
            out_col = f"ACCmag_{name}"
            cols[out_col] = np.interp(t_uniform, kin_ts, kin_mag_df[src_col].values.astype(float))
            feature_names.append(out_col)
    elif getattr(cfg, "include_kin_acc_features", False) and getattr(cfg, "kin_strict", False):
        raise RuntimeError("Kinematics features enabled (include_kin_acc_features=True) but kin_df is missing/empty.")

    # COM channels
    com_cols = [c for c in com_df.columns if c.startswith("COM_ch")]
    for c in com_cols:
        cols[c] = np.interp(t_uniform, com_ts, com_df[c].values.astype(float))
        feature_names.append(c)

    feat_df = pd.DataFrame(cols)
    return t_uniform, feat_df, feature_names

# -------------------------
# Missing-data reporting / filling (used by inference; optional elsewhere)
# -------------------------
def raw_gap_report(df: pd.DataFrame, name: str, top_n: int = 50):
    ts = df["timestamp"].values.astype(float)
    if len(ts) < 3:
        return
    dt = np.diff(ts)
    med = np.median(dt[dt > 0]) if np.any(dt > 0) else np.nan
    if not np.isfinite(med) or med <= 0:
        return
    gap_idx = np.where(dt > 5 * med)[0]
    if len(gap_idx) == 0:
        return
    print(f"\n[RAW GAP] {name}: {len(gap_idx)} gaps (dt > 5x median)")
    for k in gap_idx[:top_n]:
        t0, t1 = ts[k], ts[k+1]
        print(f"  gap: {t0:.3f} -> {t1:.3f}  (dt={t1-t0:.3f}s)")
    if len(gap_idx) > top_n:
        print(f"  ... (showing first {top_n})")

def _find_nan_segments(mask: np.ndarray, t: np.ndarray):
    mask = mask.astype(bool)
    if mask.sum() == 0:
        return []
    idx = np.where(mask)[0]
    segs = []
    start = idx[0]
    prev = idx[0]
    for i in idx[1:]:
        if i == prev + 1:
            prev = i
        else:
            segs.append((t[start], t[prev], float(t[prev] - t[start])))
            start = i
            prev = i
    segs.append((t[start], t[prev], float(t[prev] - t[start])))
    return segs

def fill_short_gaps_per_column(df_feat: pd.DataFrame, time_col: str, max_gap_sec: float, fs: float, print_top_n: int = 50):
    t = df_feat[time_col].values.astype(float)
    dt = 1.0 / fs
    max_gap_samples = int(round(max_gap_sec / dt))

    cols = [c for c in df_feat.columns if c != time_col]

    print("\n[MISSING CHECK] On uniform grid BEFORE filling:")
    printed = 0
    for c in cols:
        x = df_feat[c].values
        bad = ~np.isfinite(x)
        if bad.sum() == 0:
            continue
        segs = _find_nan_segments(bad, t)
        for (ts, te, dur) in segs:
            n_samp = int(round((te - ts) / dt)) + 1
            print(f"  {c:14s} NaN: {ts:.3f}->{te:.3f} (~{dur:.3f}s, ~{n_samp} samples)")
            printed += 1
            if printed >= print_top_n:
                print(f"  ... (showing first {print_top_n} segments total)")
                break
        if printed >= print_top_n:
            break

    df_filled = df_feat.copy()
    for c in cols:
        s = pd.Series(df_filled[c].values)
        s_interp = s.interpolate(method="linear", limit=max_gap_samples, limit_direction="both")
        df_filled[c] = s_interp.values

    if not np.isfinite(df_filled[cols].values).all():
        print("\n[WARN] NaNs remain after short-gap interpolation (gaps > MAX_GAP_SEC).")
        print("       Applying ffill/bfill so output has NO missing values.")
        df_filled[cols] = df_filled[cols].ffill().bfill()

    if not np.isfinite(df_filled[cols].values).all():
        raise RuntimeError("Still found non-finite values after filling.")

    print("[OK] All features are finite after filling.")
    return df_filled

# -------------------------
# Misstep Excel + frame->time mapping (builder use)
# -------------------------
def load_missteps_from_excel(xlsx_path: str):
    df = pd.read_excel(xlsx_path, header=None)
    misstep_dict = {}
    trial_list = []

    for i in range(df.shape[0]):
        trial_raw = df.iloc[i, 0]
        trial = normalize_trial_name(trial_raw)
        if not trial:
            continue

        events = []
        c = 1
        while c < df.shape[1]:
            frame = parse_float_from_cell(df.iloc[i, c]) if c < df.shape[1] else np.nan
            typ = parse_type_from_cell(df.iloc[i, c + 1]) if (c + 1) < df.shape[1] else "UNK"
            if np.isfinite(frame):
                events.append({"frame": float(frame), "type": typ})
            c += 2

        if trial not in misstep_dict:
            trial_list.append(trial)
            misstep_dict[trial] = events
        else:
            misstep_dict[trial].extend(events)

    for k in misstep_dict:
        misstep_dict[k] = sorted(misstep_dict[k], key=lambda d: d["frame"])

    return misstep_dict, trial_list

def map_frames_to_event_intervals(video_df: pd.DataFrame, mos_ts0: float, missteps: list, cfg: PreprocConfig):
    vid_frame = video_df["frame"].values.astype(float)
    vid_sec = video_df["t_video_s"].values.astype(float)

    time_offset = mos_ts0 - vid_sec[0]
    vid_sec_aligned = vid_sec + time_offset

    intervals = []
    for ev in missteps:
        f = float(ev["frame"])
        typ = ev.get("type", "UNK")
        idx = int(np.argmin(np.abs(vid_frame - f)))
        t_event = float(vid_sec_aligned[idx])
        t_start = t_event - float(cfg.event_tol_pre_sec)
        t_end   = t_event + float(cfg.event_tol_post_sec)
        intervals.append((t_start, t_end, typ))
    return intervals

print("[OK] preprocess_shared loaded.")


[OK] preprocess_shared loaded.
